In [14]:
# Import libraries
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()
import subprocess
import ollama
import re

In [15]:
df = pd.read_csv('./df_PreProc.csv', index_col='Unnamed: 0')
df.index.name='index'
df = df.fillna("")

indices_eq = df.index[df["description"] == df["designation"]]
df.loc[indices_eq, "description"] = " "
for i in df.index:
    if i not in indices_eq:
        df.at[i, "Designation_Description"] = df.at[i, 'designation'] + " : " + df.at[i, 'description']
    else:
        df.at[i, "Designation_Description"] = df.at[i, 'description']

df.head()


designation  \
index                                                      
0      Olivia: Personalisiertes Notizbuch / 150 Seite...   
1      Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...   
2      Grand Stylet Ergonomique Bleu Gamepad Nintendo...   
3      Peluche Donald - Europe - Disneyland 2000 (Mar...   
4                                   La Guerre Des Tuques   

                                             description   productid  \
index                                                                  
0                                                         3804725264   
1                                                          436067568   
2      PILOT STYLE Touch Pen de marque Speedlink est ...   201115110   
3                                                           50418756   
4      Luc a des idées de grandeur. Il veut organiser...   278535884   

          imageid  prdtypecode  \
index                            
0      1263597046           10   
1      1008141237         2280   
2       938777978           50   
3       457047496         1280   
4      1077757786         2705   

                                 Designation_Description  nChar  nWords  nSent  
index                                                                           
0      Olivia: Personalisiertes Notizbuch / 150 Seite...     89      14      0  
1      Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...    207      39      1  
2      Grand Stylet Ergonomique Bleu Gamepad Nintendo...    753     118      3  
3      Peluche Donald - Europe - Disneyland 2000 (Mar...     64      10      0  
4      La Guerre Des Tuques : Luc a des idées de gran...    209      38      3

#### Function LLM row wise translation

In [ ]:
# Enable tqdm progress bars for pandas apply

## Here the text from data frame translated row-wise 

tqdm.pandas()

def ollama_translate_df(df, columns, outCol, inPrompt, model):

    def ollama_translate(inText, prompt=inPrompt):
        full_prompt = prompt + f"Input Text: {inText}"
        response = ollama.chat(model=model, messages=[{"role": "user", "content": full_prompt}])
        return response["message"]["content"].strip()

    df = df.copy()
    for col in columns:
        if col not in df.columns:
            print(f"Column '{col}' not found in DataFrame. Skipping.")
            continue
        

        new_col = outCol
        df[new_col] = df.progress_apply(
            lambda row: ollama_translate(row[col]),axis=1)
        #df[new_col] = [ollama_translate(text) for text in tqdm(df[col], desc=f"Translating {col}")]


    return df

##### Split the dataFrame approach
* Here the dataframe is splitted into certain blocks
* LLM translate the rows from each blocks in series 

In [ ]:
## Here the dataframe is splitted into certain blocks
## LLM translate the rows from each blocks in series 

tqdm.pandas()

def ollama_translate2(df, columns, outCol, inPrompt, model, batch_size=2):

    def ollama_translate_batch2(inText, prompt=inPrompt):
        full_prompt = (
            f"{prompt}\n"
            f"Translate the following texts:\n{batch_text}\n"
        )
        response = ollama.chat(model=model, messages=[{"role": "user", "content": full_prompt}])
        return response["message"]["content"].strip()
    
    """
    Translate specified columns in a DataFrame using ollama in batches. 
    """
    df = df.copy()
    for col in columns:
        if col not in df.columns:
            print(f"Warning : Column '{col}' not found. Skipping..")
            continue

        new_col = f"{col}_en"
        df[new_col] = None

        n_batches = (len(df)+batch_size-1)//batch_size
        print(f"Translating column '{col}' in {n_batches} batches...")

        for batch_number, batch_df in tqdm(df.groupby(np.arange(len(df))//batch_size)):
            translated_batch = batch_df.progress_apply(lambda row: ollama_translate_batch2(row[col]),axis=1)
            df.loc[batch_df.index, new_col] = translated_batch

    return df


**Prompt to run function**

In [9]:
prompt1 = """For each text input, respond with:
- English Translation
- Max 60 words
- Only factual attributes (product type, features, notable characteristics)
- No marketing fluff, no repetition. ".
"""
inPrompt = prompt1 #+ prompt2 #+ prompt3 

df_new = df.head(20)

df_new = ollama_translate2(
    df=df_new, 
    columns=["Designation_Description"], 
    outCol="LLM_translate", 
    inPrompt=inPrompt, 
    model="llama3.2:3b",
    batch_size=32
    )

df_new.to_csv("./df_Translate.csv")

Translating column 'Designation_Description' in 1 batches...


  0%|          | 0/1 [00:00<?, ?it/s]


NameError: name 'batch_text' is not defined

#### Function LLM - Batch translation
* Data frame is splitted into few blocks as per the batch size. 
* text from that block is combined and send to llm in single prompt text. 

In [ ]:
tqdm.pandas()


def ollama_translate_batch3(inText, model, prompt):

    batch_text = ";\n".join([f"{i+1}.. {t}" for i, t in enumerate(inText)])
    full_prompt = (
        f"{prompt}\n"
        f"Translate the following texts:\n{batch_text}\n"
    )

    #print(full_prompt)

    response = ollama.chat(model=model, messages=[{"role": "user", "content": full_prompt}])
    content = response["message"]["content"].strip()


    # Split using delimiter

    #content = re.split(r'\n\d+\.', content)[1:] 


    return content

    

def ollama_translate3(df, columns, outCol, inPrompt, model, batch_size=2):

    """
    Translate specified columns in a DataFrame using ollama in batches. 
    """
    df = df.copy()
    with open("translated_output.txt", "w", encoding="utf-8") as f:
        f.flush()
        for col in columns:
            if col not in df.columns:
                print(f"Warning : Column '{col}' not found. Skipping..")
                continue

            new_col = f"{col}_en"
            df[new_col] = None

            n_batches = (len(df)+batch_size-1)//batch_size
            print(f"Translating column '{col}' in {n_batches} batches...")
            
            for batch_number, batch_df in tqdm(df.groupby(np.arange(len(df))//batch_size)):
                try:
                    batch_texts = batch_df[col].astype(str).tolist()
                    translated = ollama_translate_batch3(batch_texts, model, inPrompt)
                    #print(translated)
                    f.write("-"*10 + "\n" + "Batch Number " + str(batch_number)+"\n" + "-"*10 + "\n")
                    f.write(translated + "\n")
                    f.flush()
                    #df.loc[batch_df.index, new_col] = translated

                except Exception as e:
                    print(f"Error in batch {batch_number}: {e}")

    return df


In [18]:
prompt1 = """For each text input, respond with:
- English Translation
- Max 60 words
- Only factual attributes (product type, features, notable characteristics)
- No marketing fluff, no repetition. ".
"""
#prompt3 = f"Summarize the text with description, if it is product of review, kind of product, features, characteristic"
#prompt2 = f"the text description should not be repeated\n\n"
inPrompt = prompt1 #+ prompt2 #+ prompt3 

df_new = df.head(200)

df_new = ollama_translate3(
    df=df_new, 
    columns=["Designation_Description"], 
    outCol="LLM_translate", 
    inPrompt=inPrompt, 
    model="llama3.2:3b",
    batch_size=20
    )

#df_new[["Designation_Description", "Designation_Description_en"]].to_csv("./df_Translate.csv", index=True)


Translating column 'Designation_Description' in 10 batches...


100%|██████████| 10/10 [05:59<00:00, 36.00s/it]
